In [60]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from scipy.stats import percentileofscore
from matplotlib.gridspec import GridSpec
import matplotlib.colors as mcolors
from oauth2client.service_account import ServiceAccountCredentials
import gspread
from matplotlib.backends.backend_pdf import PdfPages
from datetime import datetime
from matplotlib.lines import Line2D
import matplotlib as mpl
from matplotlib import font_manager
mpl.rcParams.update(mpl.rcParamsDefault)

# Define the scope and load credentials
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
credentials_path = "C:/Users/benoi/OneDrive/Desktop/bea/json credentials/bea-data-7dda3770b44f.json"
creds = ServiceAccountCredentials.from_json_keyfile_name(credentials_path, scope)

# Authenticate and connect to Google Sheets
client = gspread.authorize(creds)
sheet = client.open_by_url('https://docs.google.com/spreadsheets/d/1OfgrNbgZjwMMAuVdIhOLywXUHEw1xHzYaN8ZU2awsSs/edit?gid=1169755047')
worksheet = sheet.get_worksheet(0)
data = pd.DataFrame(worksheet.get_all_records())

def generate_cover_page(df, athlete_id, pdf):
    # Get athlete details
    athlete_data = df[df['ID'] == athlete_id]
    if athlete_data.empty:
        athlete_name = f"Athlete {athlete_id}"
        status = "N/A"
        dob = "N/A"
        level = "N/A"
        school_org = "N/A"
        grad_year = "N/A"
        position = "N/A"
        weight = "N/A"
        height = "N/A"
    else:
        athlete_name = f"{athlete_data['First Name'].iloc[0]} {athlete_data['Last Name'].iloc[0]}"
        status = athlete_data['Status'].iloc[0]
        dob = athlete_data['DOB'].iloc[0] if 'DOB' in athlete_data.columns else "N/A"
        level = athlete_data['Level'].iloc[0] if 'Level' in athlete_data.columns else "N/A"
        school_org = athlete_data['School/Org'].iloc[0] if 'School/Org' in athlete_data.columns else "N/A"
        grad_year = athlete_data['Grad Year'].iloc[0] if 'Grad Year' in athlete_data.columns else "N/A"
        position = athlete_data['Position'].iloc[0] if 'Position' in athlete_data.columns else "N/A"
        
        # Get the most recent Weigh-in data
        weigh_in_data = athlete_data[athlete_data['Test Type'] == 'Weigh-in']
        if not weigh_in_data.empty:
            weight_data = weigh_in_data[weigh_in_data['Test Sub-Type'] == 'Weigh-in']
            height_data = weigh_in_data[weigh_in_data['Test Sub-Type'] == 'Height']
            weight = weight_data['Weight'].iloc[-1] if not weight_data.empty else "N/A"
            height = height_data['Height'].iloc[-1] if not height_data.empty else "N/A"
        else:
            weight = "N/A"
            height = "N/A"

    # Get the current date
    created_date = datetime.now().strftime("%B %d, %Y")

    # Create the figure for the cover page
    fig, ax = plt.subplots(figsize=(8.5, 11))  # Letter size
    ax.axis('off')  # Remove axes for a clean design

    # Title
    ax.text(
        0.5, 0.85, "Beimel Elite Athletics\nSCOPE Report",
        fontsize=30, fontweight='bold', ha='center', va='center'
    )

    # Acronym breakdown aligned vertically on the left
    acronym_text = (
        "S.trength\n"
        "C.onsistency\n"
        "O.ptimization\n"
        "P.reparation\n"
        "E.ffort"
    )
    ax.text(
        0.4, 0.65, acronym_text,
        fontsize=14, fontweight='regular', ha='left', va='center', color="gray"
    )

    # Athlete details
    details_text = (
        f"Athlete: {athlete_name}\n"
        f"Membership: {status}\n"
        f"DOB: {dob}    H/W: {height}\"/{weight}lbs \n"
        f"{school_org}  {grad_year}\n"
        f"Position: {position}\n"
        f"Created Date: {created_date}"
    )
    ax.text(
        0.5, 0.4, details_text,
        fontsize=12, fontweight='regular', ha='center', va='center'
    )

    # Footer with subtle design
    ax.text(
        0.5, 0.1, "BEA Performance Analytics",
        fontsize=10, ha='center', va='center', color="gray", alpha=0.7
    )

    # Save the figure to the PDF
    pdf.savefig(fig)
    plt.close(fig)

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
from matplotlib.backends.backend_pdf import PdfPages
from scipy.stats import percentileofscore

# Modular group-based metric definition
metric_groups = {
    'Strength': [
        ('Back Squat', 'Starting Strength', ['Weight', 'Speed']),
        ('Back Squat', 'Strength-Speed', ['Weight', 'Speed']),
        ('Back Squat', 'Accelerative Strength', ['Weight', 'Speed']),
        ('Back Squat', 'Absolute Strength', ['Weight', 'Speed']),
        ('Bench Press', 'Starting Strength', ['Weight', 'Speed']),
        ('Bench Press', 'Strength-Speed', ['Weight', 'Speed']),
        ('Bench Press', 'Accelerative Strength', ['Weight', 'Speed']),
        ('Bench Press', 'Absolute Strength', ['Weight', 'Speed']),
        ('Deadlift', 'Starting Strength', ['Weight', 'Speed']),
        ('Deadlift', 'Strength-Speed', ['Weight', 'Speed']),
        ('Deadlift', 'Accelerative Strength', ['Weight', 'Speed']),
        ('Deadlift', 'Absolute Strength', ['Weight', 'Speed']),
        ('Grip', 'Arm Side', ['Weight']),
        ('Grip', 'Glove Side', ['Weight']),
        ('Jump', 'Broad', ['Distance']),
        ('Jump', 'Lateral Block Leg', ['Distance']),
        ('Jump', 'Lateral Load Leg', ['Distance']),
        ('Jump', 'Vertical', ['Vert']),
        ('Jump', 'Vertical Block Leg', ['Vert']),
        ('Jump', 'Vertical Load Leg', ['Vert']),
        ('Weigh-in', 'Weigh-in', ['Weight']),
        ('Weigh-in', 'Height', ['Height'])
    ],
    'Plyos': [
        *[(t, w, ['Max Velo', 'Average Velo']) for t in ['Roll Ins', 'Double Plays', 'Turn and Burns', 'Pulldowns', 'Catchers Velo', 'Mound Velo']
          for w in ['3oz', '4oz', '5oz', '6oz', '7oz']]
    ],
    'Pitching - Trackman': [
        ('Trackman Bullpen', 'Fastball', [
            'Max Velo', 'Average Velo', 'Max Spin', 'Average Spin',
            'IVB', 'HB', 'Extension', 'Rel Height (ft)', 'Rel Side (ft)',
            'Gyro', 'VAA', 'Total Strike %'
        ])
    ],
    'ArmCare': [
        ('ArmCare', 'Fresh Exam', [
            'Arm Score', 'Total Strength', 'Shoulder Balance', 'SVR',
            'IR ROM', 'ER ROM', 'Flexion ROM'
        ])
    ],
    'Hitting': [
        *[(t, sub, ['Average EV', 'Max EV', 'Max Distance']) for t, sub in [
            ('HitTrax', 'Tee'), ('HitTrax', 'Front Toss'), ('HitTrax', 'Machine/BP'), ('HitTrax', 'Live AB')]],
        *[(t, sub, ['Max Bat Speed', 'Peak Hand Speed']) for t, sub in [
            ('Blast Motion', 'Tee'), ('Blast Motion', 'Front Toss'), ('Blast Motion', 'Machine/BP'), ('Blast Motion', 'Live AB')]]
    ]
}

# Flattened metric_dict for percentiles
metric_dict = {}
for group in metric_groups.values():
    for test_type, sub_type, metrics in group:
        metric_dict.setdefault((test_type, sub_type), []).extend(metrics)

# Unit dictionary by metric
unit_dict = {
    # General
    'Weight': 'lbs',
    'Speed': 'm/s',
    'Distance': 'feet',
    'Vert': 'inches',
    'Height': 'inches',

    # Pitching
    'Max Velo': 'MPH',
    'Average Velo': 'MPH',
    'Max Spin': 'RPM',
    'Average Spin': 'RPM',
    'IVB': 'inches',
    'HB': 'inches',
    'Extension': 'ft',
    'Rel Height (ft)': 'ft',
    'Rel Side (ft)': 'ft',
    'Gyro': '°',
    'VAA': '°',
    'Total Strike %': '%',

    # Hitting
    'Max Bat Speed': 'MPH',
    'Peak Hand Speed': 'MPH',
    'Average EV': 'MPH',
    'Max EV': 'MPH',
    'Max Distance': 'ft',

    # ArmCare
    'Arm Score': '',
    'Total Strength': '',
    'Shoulder Balance': '',
    'SVR': '',
    'IR ROM': '°',
    'ER ROM': '°',
    'Flexion ROM': '°'
}


def calculate_percentiles(df, athlete_id, metric_dict, verbose=False):
    """
    Calculate percentile rankings for all test numbers for each metric for an athlete compared to peers at the same level.
    """
    from scipy.stats import percentileofscore

    athlete_data = df[df['ID'] == athlete_id]
    if athlete_data.empty:
        if verbose:
            print(f"No data for athlete ID {athlete_id}")
        return pd.DataFrame(), None, None

    athlete_name = f"{athlete_data['First Name'].iloc[0]} {athlete_data['Last Name'].iloc[0]}"
    athlete_level = athlete_data['Level'].iloc[-1]
    peer_data = df[df['Level'] == athlete_level]

    records = []

    for (test_type, sub_type), metrics in metric_dict.items():
        for metric in metrics:
            subset = peer_data[
                (peer_data['Test Type'] == test_type) &
                (peer_data['Test Sub-Type'] == sub_type) &
                (peer_data[metric].notna())
            ].copy()

            subset[metric] = pd.to_numeric(subset[metric], errors='coerce')
            subset = subset.dropna(subset=[metric])
            if subset.empty:
                continue

            athlete_metric_data = athlete_data[
                (athlete_data['Test Type'] == test_type) &
                (athlete_data['Test Sub-Type'] == sub_type) &
                (athlete_data[metric].notna())
            ].copy()

            athlete_metric_data[metric] = pd.to_numeric(athlete_metric_data[metric], errors='coerce')
            athlete_metric_data = athlete_metric_data.dropna(subset=[metric])

            for _, row in athlete_metric_data.iterrows():
                test_number = row.get('Test Number', None)
                test_date = row.get('Date', None)
                test_value = row[metric]
                percentile = percentileofscore(subset[metric], test_value)

                records.append({
                    'Test Type': test_type,
                    'Sub-Type': sub_type,
                    'Metric': metric,
                    'Test Number': test_number,
                    'Date': test_date,
                    'Value': test_value,
                    'Percentile': percentile,
                    'Athlete ID': athlete_id,
                    'Level': athlete_level
                })

                if verbose:
                    print(f"{test_type} - {sub_type} - {metric} | Test #{test_number} | {test_date}: Value={test_value} → Percentile={percentile:.1f}%")

    results_df = pd.DataFrame(records)
    return results_df, athlete_name, athlete_level


def plot_savant_chart(percentile_df, athlete_name, pdf):
    from matplotlib.colors import LinearSegmentedColormap

    # Custom colormap from blue (low) to red (high)
    custom_cmap = LinearSegmentedColormap.from_list("BlueRed", ["#2c7bb6", "#abd9e9", "#ffffbf", "#fdae61", "#d7191c"])

    categories = metric_groups.keys()
    for category in categories:
        rows = []
        labels_in_order = []

        for test_type, sub_type, metrics in metric_groups[category]:
            # Skip non-Strength-Speed sub-types for lifts
            if category == 'Strength' and test_type in ['Back Squat', 'Bench Press', 'Deadlift'] and sub_type != 'Strength-Speed':
                continue

            for metric in metrics:
                filtered = percentile_df[
                    (percentile_df['Test Type'] == test_type) &
                    (percentile_df['Sub-Type'] == sub_type) &
                    (percentile_df['Metric'] == metric)
                ]
                if not filtered.empty:
                    most_recent = filtered.sort_values(by='Test Number', ascending=False).iloc[0].copy()
                    most_recent['Metric Label'] = f"{test_type} - {sub_type} - {metric}"
                    most_recent['Display Value'] = most_recent['Value']
                    most_recent['Display Percentile'] = most_recent['Percentile']
                    rows.append(most_recent)
                    labels_in_order.append(most_recent['Metric Label'])

        if not rows:
            continue

        cat_df = pd.DataFrame(rows)
        cat_df.set_index("Metric Label", inplace=True)
        cat_df = cat_df.loc[labels_in_order[:len(cat_df)]]

        percentiles = cat_df['Display Percentile']
        values = cat_df['Display Value']
        metrics = cat_df['Metric']
        y_labels = cat_df.index.tolist()

        # Fixed bar height and consistent spacing
        bar_height = 0.4
        fig_height = bar_height * len(y_labels) + 2
        fig, ax = plt.subplots(figsize=(10, fig_height))

        # Normalize for colormap
        norm = plt.Normalize(0, 100)
        bar_colors = custom_cmap(norm(percentiles))

        # Background track bars (full length)
        ax.barh(y_labels, [100]*len(y_labels), color='#f0f0f0', height=bar_height*.4, zorder=0)

        # Actual performance bars
        bars = ax.barh(y_labels, percentiles, color=bar_colors, height=bar_height, zorder=1)

        for i, (bar, perc, val, met) in enumerate(zip(bars, percentiles, values, metrics)):
            unit = unit_dict.get(met, '')
            bar_center = bar.get_y() + bar.get_height() / 2
            badge_color = bar_colors[i]

            ax.text(
                bar.get_width(), bar_center, f"{int(perc)}",
                ha='right', va='center', fontsize=8, color='white', fontweight='bold',
                bbox=dict(boxstyle='circle', fc=badge_color, ec='white', linewidth=1.0, alpha=0.95),
                zorder=2
            )

            ax.text(104, bar_center, f"{val:.1f} {unit}", va='center', ha='left', fontsize=8)

        ax.set_xlim(0, 110)
        ax.set_xticks([])
        ax.set_xlabel('')
        ax.set_title(f"{athlete_name} - {category}", fontweight='bold', fontsize=14)
        ax.invert_yaxis()
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.spines['left'].set_visible(False)
        ax.spines['bottom'].set_visible(False)
        ax.tick_params(axis='y', labelsize=9)

        plt.tight_layout()
        pdf.savefig(fig)
        plt.close()


def generate_scope_report(df, athlete_id, save_directory):
    """
    Generate a sequential, modular SCOPE 2.0 progress report for an athlete.

    Args:
        df (pd.DataFrame): Full athlete data from Google Sheets.
        athlete_id (int): Athlete ID to generate report for.
        save_directory (str): Folder to save the PDF output.
    """

    athlete_data = df[df['ID'] == athlete_id]
    if athlete_data.empty:
        print(f"No data found for athlete ID {athlete_id}.")
        return

    # Get name
    athlete_name = f"{athlete_data['First Name'].iloc[0]} {athlete_data['Last Name'].iloc[0]}"

    # Get most recent strength workout phase
    strength_data = athlete_data[athlete_data['Workout Type'] == 'Strength']
    if not strength_data.empty:
        strength_data_sorted = strength_data.sort_values(by=['Test Number', 'Date'], ascending=[False, False])
        recent_strength = strength_data_sorted.iloc[0]
        phase_name = recent_strength.get('Phase', '')
    else:
        phase_name = ''

    # Build file name and path
    today = datetime.now().strftime("%Y-%m-%d")
    file_name = f"{athlete_name}, {phase_name}, {today}.pdf"
    pdf_path = f"{save_directory}/{file_name}"

    # ➕ Calculate Percentiles
    percentile_df, athlete_name, athlete_level = calculate_percentiles(df, athlete_id, metric_dict)

    # Start PDF report
    with PdfPages(pdf_path) as pdf:
        print(f"Initialized report for {athlete_name} — saving to {pdf_path}")
        
        # Future steps: Call your custom functions here
        generate_cover_page(df, athlete_id, pdf)
        
        plot_savant_chart(percentile_df, athlete_name, pdf)
        pass

    print(f"SCOPE 2.0 report saved to: {pdf_path}")

# 📝 Manually enter athlete ID and directory here
athlete_id = 112109
save_directory = 'C:/Users/benoi/OneDrive/Desktop/bea/SCOPE_Reports'

# 📄 Generate the report
generate_scope_report(df, athlete_id, save_directory)





Initialized report for Matthew Coye — saving to C:/Users/benoi/OneDrive/Desktop/bea/SCOPE_Reports/Matthew Coye, Phase 3, 2025-04-11.pdf
SCOPE 2.0 report saved to: C:/Users/benoi/OneDrive/Desktop/bea/SCOPE_Reports/Matthew Coye, Phase 3, 2025-04-11.pdf
